# Install modules

In [31]:
! pip install numpy
! pip install Pillow

# multiprocess module works, unlike conturent.futures.ProcessPoolExecutor,
#   with IPython (jupyter)
! pip install multiprocess

# wikipedia module used for donwloading texts from Wikipedia
! pip install wikipedia


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


# 1. Scrape Source Texts from Wikipedia

In [4]:
import wikipedia
import re
import urllib.request
import time
import json
from concurrent import futures
import threading
from pathlib import Path

with urllib.request.urlopen('https://simple.wikipedia.org/wiki/List_of_countries') as f:
    scraped = f.read(1000000)

countries = re.findall(r'<a.*?title="(.*?)">.*?<\/a>', scraped.decode('utf-8'))
start = countries.index('Afghanistan')
end_incl = countries.index('Zimbabwe')
countries = countries[start: end_incl+1]

excluded_data = ['Change section']
def is_excluded(country):
    return any(e in country for e in excluded_data)

countries = [country for country in countries 
             if not is_excluded(country)] 

lock = threading.Lock()
remaining = len(countries)

def wiki_download(name):
    text = wikipedia.page(name, redirect=False, auto_suggest=False).content
    text = re.sub(r'==.*?==+', '', text)
    text = text.replace('\n\n\n', '\n')
    text = text.strip().split()
    
    global remaining
    with lock:
        remaining -= 1
        print(f'\r remaining: {remaining:03d} ', end='', flush=True)
        
    return text

# multithreading reduces download time 
# from > 3 minutes (single-thread version) to < 20s
with futures.ThreadPoolExecutor(16) as executor:
    texts = executor.map(wiki_download, countries)

# texts is a generator: list(texts) needed
with open(Path('data_scraped') / 'scraped_texts.json', 'wt') as f:
    json.dump(list(texts), f, ensure_ascii=False)

 remaining: 000 

# 2. Generate Image

- all scans need to have the same resolution (upscaling/downscaling)
- potential extension: extracting numbers from tables
- line breaks

In [1]:
from xml.dom.minidom import Text
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import math
import random
import json
from typing import Iterable, Union, Tuple, List, Iterable, Generator
from pathlib import Path
import numpy as np
import multiprocess

COLOR = (250, 250, 250)
PAPER_WIDTH = 800
MIN_TEXT_SIZE = 14  # can be increased with higher resolution (paper width) 
MAX_TEXT_SIZE = 20

CoordinatesRect = Tuple[Tuple[int, int], Tuple[int, int]]

STANDARD_BLOCK = dict(
    width_frac_interval=(0.2, 0.5),
    height_frac_interval=(0.2, 0.5),
    margin=(0.07, 0.07)
)

JUST_NUMBER_BLOCK = dict(
    width_frac_interval = (0.15, 0.15),
    height_frac_interval = (0.1, 0.1),
    margin = STANDARD_BLOCK['margin']
)

MAX_ATTEMPTS = 500

class GenerateImage:
    def __init__(self, text_split, num_idx=-1, 
                 paper_width = PAPER_WIDTH, ratio = 2 ** 0.5) -> None:
        self.rng = np.random.default_rng()
        self.text_split = text_split
        self.paper_width = paper_width
        self.paper_height = int(paper_width * ratio)
        self.num_idx = num_idx
        self.font_families = self.get_fonts()
        self.font_variants = 'Regular Italic Bold BoldItalic'.split()
        self.reset_image()
        
    def reset_image(self):
        self.image = Image.new('RGB', size=(self.paper_width, self.paper_height), 
                color=COLOR)  # A4
        self.draw = ImageDraw.Draw(self.image)

    def get_fonts(self):
        """ Get unique names of fonts in the font library 
        (w/o regular/italic/bold/bolditalic variants """
        # wrapped in list since sets (unlike dicts as from python 3.7) are not guaranteed
        # to be stable and we need to index font families (while choosing at random)
        return list({entry.name.split('-')[0] 
                    for entry in Path('fonts').iterdir() if entry.is_file()})
        
    @staticmethod
    def value_in_interval(theta: float, imin: float, imax: float):
        assert imax >= imin
        assert 0 <= theta <= 1
        return imin + (imax - imin) * theta

    def get_block_rectangle(self, 
                              width_frac_interval: Tuple[float, float],
                              height_frac_interval: Tuple[float, float],
                              margin: Tuple[float, float]
                              ) -> CoordinatesRect:
        """Generates a random rectangle (for the text to be placed in) according to
        the input parameters. Does not check any interesections.
        Args:
            width_frac_interval: min and max width (e.g. 0.3 == 30 % of the page width)
            height_frac_interval: min and max height (e.g. 0.4 == 40 % of the page heigth)
            margin (Tuple[float, float]): margin in 
        Returns:
            CoordinatesRect: coordinates of the rectangle
        """
        block_width_fraction = self.value_in_interval(theta=self.rng.random(),                                       
                                                      imin=width_frac_interval[0], 
                                                      imax=width_frac_interval[1])
        block_width = int(self.paper_width * block_width_fraction)

        block_height_fraction = self.value_in_interval(theta=self.rng.random(),                                       
                                                       imin=height_frac_interval[0], 
                                                       imax=height_frac_interval[1])
        block_height = int(self.paper_height * block_height_fraction)

        remaining_width = self.paper_width - block_width
        remaining_height = self.paper_height - block_height

        margin_x, margin_y = int(margin[0] * self.paper_width), int(margin[1] * self.paper_height)
        x_pos = self.rng.integers(margin_x, remaining_width - margin_x, endpoint=True)
        y_pos = self.rng.integers(margin_y, remaining_height - margin_y, endpoint=True)
        
        return (x_pos, y_pos), (x_pos+block_width, y_pos+block_height)
        
    def get_line_data(self, start_idx: int, text: List[str], font_size: int, 
                      font_family: str, block_width: int) -> Tuple[List[int], List[int]]:
        """Checks what part of the text can be printed to fit the block_width
        Args:
            start_idx (int): index in text with the first yet unprinted word
            text (List[str]): list of words (some of them could be printed on the lines(s) above)
            block_width (int): width of line in pixels

        Returns:
            Tuple[List[int], List[int]]: 
                item [0] of each of lists relates to the text[start_idx] word
                item [1] of each of lists relates to the text[start_idx+1] word
                ...
                item [-1] of each lists relates to the text[start_idx+len(lists)-1],
                    the last word that fits the line
                ...
                the first lists includes randomly chosen variants of the font 
                    (0==regular, ...), see self.font_variants
                the second output includes width of each word (including the leading
                space unless the word is the first on the line)
        """
        variants, widths = [], []
        committed_width = 0
        for idx in range(start_idx, len(text)):
            variant = self.rng.multinomial(1, [0.6] + [0.4 / 3] * 3).nonzero()[0].item()
            font_path = Path('fonts') / f'{font_family}-{self.font_variants[variant]}.ttf'
            font = ImageFont.truetype(str(font_path), font_size) 
            sep = '' if idx == start_idx else ' '
            text_width = self.draw.textlength(sep + text[idx], font=font)
            if committed_width + text_width > block_width:
                return variants, widths
            else:
                committed_width += text_width
                variants.append(variant)
                widths.append(text_width)
        # we can get here after all words from text are seen in the for loop
        return variants, widths
    
    def print_line(self, xy: Tuple[int, int], start_idx: int, text: List[str], font_size: int,
                   font_family: str, block_width: int) -> int:
        """Prints words from text (starting with the start_idx) index on the current line 
        (given by xy coordinages)
        Args:
            xy: coordinates of the top left corner of the current line
            start_idx: index in text with the first yet unprinted word
            text: list of words (some of them could be printed on the lines(s) above)
            block_width: width of line in pixels
        Returns:
            number of words printed on the current line
        """
        variants, widths = self.get_line_data(start_idx, text, 
                                              font_size, font_family, block_width)
        current_x, y = xy 
        all_text = ''
        for i, variant in enumerate(variants):
            font_path = Path('fonts') / f'{font_family}-{self.font_variants[variant]}.ttf'
            font = ImageFont.truetype(str(font_path), font_size)  # type: ignore
            sep = '' if i == 0 else ' '
            current_word = sep + text[start_idx + i]
            self.draw.text((current_x, y), current_word, 
                           fill=(40,40,40), font=font)
            all_text += current_word
            current_x += widths[i]
        
        return len(variants)
            
    def print_block(self, text: List[str], coordinates: CoordinatesRect) -> str:
        """ Prints the block of text. We cannot use the Image.Draw.multline_text method, which 
        reguires the same font for all words.
        Args:
            text: List of words. The method selects a contiguous span and prints it.
            coordinates (CoordinatesRect): the printed text will be positioned within
                this (x0, y0), (x1, y1) rectangle
        Returns:
            str: text printed in the rectangle
        """
        (x0, y0), (x1, y1) = coordinates
        block_width = x1 - x0 + 1
        block_height = y1 - y0 + 1
        
        max_start = int(0.75 * len(text))
        start_idx = 0 if len(text)==1 else self.rng.integers(0, max_start)
        current_idx = start_idx
        
        current_y = y0
        
        font_size = self.rng.integers(MIN_TEXT_SIZE, MAX_TEXT_SIZE, endpoint=True)
        font_family_index = self.rng.integers(0, len(self.font_families))
        font_family = self.font_families[font_family_index]
        
        while True:
            add_y = int(font_size * 1.3)
            
            if current_y + add_y > y1:
                break
            
            words = self.print_line((x0, current_y), current_idx, text,      
                                    font_size, font_family, block_width)
            
            current_idx += words
            current_y += add_y
            
            # len(text) is checked also inside print_line -> get_line_data
            if words == 0 or current_idx >= len(text):
                break
            
        return ' '.join(text[start_idx: current_idx])
    
    @staticmethod
    def intersects(block1: CoordinatesRect, block2: CoordinatesRect,
                   margin: Tuple[int, int]) -> bool:
        """checks whether the two blocks intersect"""
        ((ax0, ay0), (ax1, ay1)) = block1
        ((bx0, by0), (bx1, by1)) = block2
        margin_x, margin_y = margin
        bx0, by0 = bx0 - margin_x, by0 - margin_y
        bx1, by1 = bx1 + margin_x, by1 + margin_y
        # rectange A is fully to the left or fully to the right of B
        if (ax0 < bx0 and ax1 < bx0) or (ax0 > bx1 and ax1 > bx1):
            return False
        # rectange A is fully to the top or fully to bottom of B
        if (ay0 < by0 and ay1 < by0) or (ay0 > by1 and ay1 > by1):
            return False
        return True
    
    def get_standard_blocks_coordinates(self, 
                                        no_of_blocks: int) -> List[CoordinatesRect]:
        """ Returns the list of up to no_of_block rectangles, without any overlap.
        Please excuse the quick solution based on placing rectangles randomly and 
        checking for overlaps.
        """
        assert no_of_blocks >= 1
        all_coordinates = []
        for _ in range(MAX_ATTEMPTS):  # the maximum number of attempts
            new_coords = self.get_block_rectangle(**STANDARD_BLOCK)
            margin = (int(STANDARD_BLOCK['margin'][0] * self.paper_width),
                      int(STANDARD_BLOCK['margin'][1] * self.paper_height))
            # any([]) == False
            if any(self.intersects(new_coords, old_coords, margin)  
                   for old_coords in all_coordinates):
                continue
            all_coordinates.append(new_coords)
            if len(all_coordinates) == no_of_blocks:
                return all_coordinates
        assert len(all_coordinates) >= 1
        return all_coordinates
    
    def get_number_block_coordinates(self, all_coordinates: 
                                     List[CoordinatesRect]) -> CoordinatesRect:
        """find another smaller rectangle, to include just a single number, that
        does not collide with any of the all_coordinates rectangles
        """
        for _ in range(MAX_ATTEMPTS * 10):
            coords = self.get_block_rectangle(**JUST_NUMBER_BLOCK)
            margin = (int(JUST_NUMBER_BLOCK['margin'][0] * self.paper_width),
                      int(JUST_NUMBER_BLOCK['margin'][1] * self.paper_height))
            if not any(self.intersects(coords, old_coords, margin)  
                       for old_coords in all_coordinates):
                return coords
        raise Exception("Unable to place the number block")
        
    def get_random_number_string(self) -> str:
        '''get a random number (as text) to be placed in to the numeric rectangle'''
        number = self.rng.integers(10000, 999999, endpoint=True)
        variant = self.rng.integers(0, 3)
        if variant == 0:
            text = str(number)
        elif variant == 2:
            text = '{:,}'.format(number)
        else:
            text = '{:,}'.format(number).replace(',', ' ')
        return text
    
    @staticmethod
    def make_result_dict(blockText: str, coordinates: CoordinatesRect):
        return dict(text=blockText, 
                    x0=int(coordinates[0][0]),
                    y0=int(coordinates[0][1]),
                    x1=int(coordinates[1][0]),
                    y1=int(coordinates[1][1]))
    
    def print_blocks(self, no_of_standard_blocks: int) -> List:
        """ Prints a numeric block and no_of_standard_blocks standard blocks.
        Returns: A list regarding printed out blocks. Item[0] always relates to 
        the rectangle with a single number. Each item is a dict including value 
        (the text in the rectangle) and coordinates (x0, y0, x1, y1 attributes) 
        of the rectangle.
        """
        while True:  # loop until numeric block placed
            standard_results = []
            std_coordinates = self.get_standard_blocks_coordinates(no_of_standard_blocks)
            for coords in std_coordinates:
                text_idx = self.rng.integers(0, len(self.text_split))
                text = self.text_split[text_idx]
                res = self.print_block(text, coords)
                standard_results.append(self.make_result_dict(res, coords))
            
            # add a single plain numeric block
            text = self.get_random_number_string().split()
            try:
                nb_coordinates = self.get_number_block_coordinates(std_coordinates)
                numeric_result = self.print_block(text, nb_coordinates)
                numeric_dict = self.make_result_dict(numeric_result, nb_coordinates)
                return [numeric_dict] + standard_results
            except:  # unable to place the numeric block, the while loop will continue
                self.reset_image()
        
    def process_img_deterministic(self, blur, noise, contrast, offset, rotate):
        self.image = self.image.filter(ImageFilter.GaussianBlur(radius=blur))
        noise = Image.effect_noise(self.image.size, noise)
        noise_arr = np.asarray(noise).astype(float)[:, :, None] - 128
        image_arr = np.asarray(self.image).astype(float)
        image_arr = (image_arr * contrast) + offset
        res_arr = (noise_arr + image_arr).clip(0, 255)
        self.image = Image.fromarray(res_arr.astype(np.uint8))
        self.image = self.image.rotate(rotate, 
                                       fillcolor=COLOR,
                                       resample=Image.Resampling.BICUBIC)  # type: ignore

    def process_image(self) -> float:
        rotation = self.rng.uniform(-2,2)
        self.process_img_deterministic(blur=self.rng.uniform(0, 0.3),
                                       noise=self.rng.uniform(1,5),
                                       contrast=self.rng.uniform(0.95, 1.05),
                                       offset=self.rng.uniform(-7,7),
                                       rotate=rotation)
        return rotation

    def save_image(self, path):
        self.image.save(path)
        return self
    
    def __call__(self, path, blocks: Union[int, Tuple[int, int]]):
        if isinstance(blocks, Iterable):
            assert len(blocks) == 2
            blocks = self.rng.integers(blocks[0], blocks[1], endpoint=True)
        result = self.print_blocks(blocks)
        rotation = self.process_image()
        result[0]['rotation'] = rotation
        self.save_image(path)      
        return result
    
def process_one(num_idx, display_image=False):
    with open(Path('data_scraped') / 'scraped_texts.json', 'rt') as f:
        texts = json.load(f)
    path = Path('blocks_images') / f'img{num_idx:04d}.jpg'
    genimg = GenerateImage(texts, num_idx=num_idx)
    result = genimg(path=path, blocks=(2, 4))
    if display_image:
        display(genimg.image)
    return result

# process_one(0)

from multiprocessing import cpu_count
import multiprocess

GENERATED_IMAGES_COUNT = 100

cores = cpu_count()
with multiprocess.Pool(cores) as pool:
    results = pool.map(process_one, range(GENERATED_IMAGES_COUNT))
    
for i, result in enumerate(results):
    path = Path('blocks_texts') / f'{i:04d}.json'
    with open(path, 'wt') as f:
        json.dump(result, f, ensure_ascii=False)

# 3. OCR Retrieval

The retrieval is made using the numbers_extract.html web application. Please download its results and save it as extracted_numbers.txt in the current directory (where this Python / .ipynb file is located)

# 4. Evaluation of OCR Retrieval

In [38]:
# read and parse OCR results

with open('extracted_numbers.txt', 'rt') as f:
    lines = f.readlines()
    
def splitstrip(line, char):
    splits = line.split(char)
    return [s.strip() for s in splits]
   
ocr_groups = [] 
for line in lines:
    file_name, rest = splitstrip(line, ':')
    blocks = splitstrip(rest, '|')
    blocks = [splitstrip(b, ';') for b in blocks]
    ocr_groups.append([file_name] + blocks)
    
# sort by filename
ocr_groups.sort(key=lambda item: item[0])
# print(groups)

# extract just a number of each group (for comparison with true data)
group_numbers = [item[0][3:3+4] for item in ocr_groups]
print(group_numbers)
    

['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0028', '0029', '0030', '0031', '0032', '0033', '0034', '0035', '0036', '0037', '0038', '0039', '0040', '0041', '0042', '0043', '0044', '0045', '0046', '0047', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0058', '0059']


In [39]:
# read and extract true data
from pathlib import Path

dir_name = 'blocks_texts'
path = Path(dir_name)
file_names = [str(entry)[len(dir_name)+1:] 
              for entry in path.iterdir() if entry.is_file()]
file_names.sort()
texts_file_names = [fn for fn in file_names if fn[:-5] in group_numbers]

file_numbers =[fn[:4] for fn in file_names]
if all(gn in file_numbers for gn in group_numbers):
    print('all filenames returned by the OCR retrieval are included in the local true set')
    print('names of images and local texts for evaluating the OCR quality:')
    images_file_names = [item[0] for item in ocr_groups]
    
    print(*zip(images_file_names, texts_file_names))

all filenames returned by the OCR retrieval are included in the local true set
names of images and local texts for evaluating the OCR quality:
('img0000.jpg', '0000.json') ('img0001.jpg', '0001.json') ('img0002.jpg', '0002.json') ('img0003.jpg', '0003.json') ('img0004.jpg', '0004.json') ('img0005.jpg', '0005.json') ('img0006.jpg', '0006.json') ('img0007.jpg', '0007.json') ('img0008.jpg', '0008.json') ('img0009.jpg', '0009.json') ('img0010.jpg', '0010.json') ('img0011.jpg', '0011.json') ('img0012.jpg', '0012.json') ('img0013.jpg', '0013.json') ('img0014.jpg', '0014.json') ('img0015.jpg', '0015.json') ('img0016.jpg', '0016.json') ('img0017.jpg', '0017.json') ('img0018.jpg', '0018.json') ('img0019.jpg', '0019.json') ('img0020.jpg', '0020.json') ('img0021.jpg', '0021.json') ('img0022.jpg', '0022.json') ('img0023.jpg', '0023.json') ('img0024.jpg', '0024.json') ('img0025.jpg', '0025.json') ('img0026.jpg', '0026.json') ('img0027.jpg', '0027.json') ('img0028.jpg', '0028.json') ('img0029.jpg', 

In [40]:
import json
import re

with open('blocks_texts/0000.json') as f:
    js = json.load(f)
#print(js)

def convert_to_num(s):
    return float(s) if '.' in s else int(s)

def extract_numbers(text):
    reg_string= r"\d+(?:[\s,]\d+)*(?:\.\d+)?"
    found = re.findall(reg_string, text)
    return [convert_to_num(s.replace(' ', '').replace(',', ''))
            for s in found]

real_groups = []
for text_file_name in texts_file_names:
    with open(Path('blocks_texts') / text_file_name, 'rt') as f:
        page = json.load(f)
        for block in page:
            block['numbers'] = extract_numbers(block['text'])
            block['avg_y'] = (block['y0'] + block['y1']) / 2
        # the first paragraph includes the key/index number
        # subsequent paragraphs will be sorted according to the y position
        first = page[0]
        other = page[1:]        
        other.sort(key=lambda group: group['avg_y'])    
        real_groups.append([first] + other)        

In [41]:
assert len(ocr_groups) == len(real_groups)

compare_data = []

for og, rg in zip(ocr_groups, real_groups):
    image_name = og[0]
    ocr_nums = og[1:]
    ocr_nums = [[] if len(para) == 1 and para[0] == '' else [convert_to_num(n) for n in para] 
                for para in ocr_nums]
    rg_nums = [item['numbers'] for item in rg]
    compare_data.append(dict(
        image_name=image_name, ocr_nums=ocr_nums, rg_nums=rg_nums
    ))

### Klíčová čísla dokumentů

In [43]:
# zde se měří kvalita extrakce klíčových čísel (na každé stránce je jedno samostatné číslo)

misses = [i for i in range(len(compare_data))
          if compare_data[i]['ocr_nums'][0] != compare_data[i]['rg_nums'][0]]

acc = 1 - len(misses)/len(compare_data)
print(f'accuracy: {acc*100:.2f} %')

print(f'soubor         true      predikce')
print('=========================================')
for miss_idx in misses:
    fl = compare_data[miss_idx]['image_name']
    predicted = compare_data[miss_idx]['ocr_nums'][0]
    pred_print = '; '.join(str(p) for p in predicted)
    true_num = compare_data[miss_idx]['rg_nums'][0][0]
    print(f'{fl:15s}{str(true_num):10s}{pred_print}')
    
# níže uvedený seznam umožňuje blíže analyzovat chyby (zejména se zdá, že OCR má problém s čísly, 
# která jsou současně tištěna kurzívou, menším fontem a patkovým písmem)

# patrně by bylo také třeba generovat (scanovat) dokumenty v podstatně vyšším rozlišení
# může také připadat v úvahu, že šum je na některých obrázcích nerealisticky agresivní

# k větší průkaznosti výsledků by bylo třeba provést predikce na podstatně více než 60 dokumentech

accuracy: 91.67 %
soubor         true      predikce
img0008.jpg    703123    703129
img0015.jpg    22547     22; 547
img0034.jpg    216641    216611
img0035.jpg    666434    10; 13; 1000
img0053.jpg    843562    3562


### Print-out

In [45]:
for cd in compare_data:
    print(cd['image_name'])
    print('true    ', cd['rg_nums'])
    print('predikce', cd['ocr_nums'])

img0000.jpg
true     [[857139], [50, 2013, 161, 35, 22], [33, 1990, 1993], [1046, 650, 6774, 4209, 2430, 1510, 523, 325, 2912, 1809, 909, 565]]
predikce [[857139], [50, 2013, 161, 35, 22], [33, 1990, 1993], [6, 630, 6774, 4209, 2430, 1510, 523, 325, 2912, 9, 909, 565]]
img0001.jpg
true     [[165595], [2006, 2010, 2011, 18], [847, 855, 1534, 1549, 1559, 1565, 1623, 1644, 1929], [1650, 44000, 12000, 23000, 10, 1630, 5, 10, 2, 4]]
predikce [[165595], [2006, 2010, 2011, 18, 1], [847, 855, 1534, 1549, 1559, 1565, 1623, 1644, 1929], [1650, 44, 0, 12000, 23000, 1630, 5, 10, 2]]
img0002.jpg
true     [[936359], [1972], []]
predikce [[936359], [1972], []]
img0003.jpg
true     [[928952], [2009, 1958, 1958, 1959, 1965], [5, 7, 1922, 64, 57, 6, 1922, 8, 1922], [1961]]
predikce [[928952], [2009, 1958, 1958, 1959, 1965], [1961], [5, 7, 1922, 64, 57.1, 6, 1922, 8, 1922]]
img0004.jpg
true     [[621258], [1950, 1988], [1943, 1919]]
predikce [[621258], [1950, 1988], [1943, 1919]]
img0005.jpg
true     [[5